In [1]:
# encrypt triples on closed statements.

import ansible_vault
import json
import pathlib
import rdflib
import uuid

ont = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/")
res = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/")

graph_in = rdflib.Graph()
graph_in.parse(pathlib.Path.cwd() / "data.json")
graph_in.bind("ont", ont)
graph_in.bind("res", res)

graph_out = rdflib.Graph()

keys = dict()
closed = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.ClosedStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in closed and p == ont.hasPayload:
        key = str(uuid.uuid4())
        keys[s] = key
        enc_statement = ansible_vault.Vault(key).dump_raw(o)
        graph_out.add((s, p, rdflib.Literal(enc_statement)))
    else:
        graph_out.add((s, p, o))

with open(pathlib.Path.cwd() / "keys.json", "w") as author:
    json.dump(keys, author, indent=4)

graph_out.serialize(
    destination=str(pathlib.Path.cwd() / "example.json"), format="json-ld"
)
print(len(graph_out), "triples.")


38 triples.


In [2]:
# parse open statements.

graph_in = rdflib.Graph()
graph_in.parse(pathlib.Path.cwd() / "example.json")

graph_out = rdflib.Graph()
graph_out.bind("ont", ont)
graph_out.bind("res", res)

openstate = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.OpenStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in openstate and p == ont.hasPayload:
        fragment = rdflib.Graph()
        fragment.parse(data=o)
        graph_out += fragment

print(graph_out.serialize(format="ttl"))


@prefix ont: <https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix res: <https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/> .

res:d4b77081-311b-4a44-b388-6dfb826faa0d a ont:BirthEvent ;
    ont:hasActivity res:6dfbacdc-b175-4cd4-8010-7dae5132707c .

res:6dfbacdc-b175-4cd4-8010-7dae5132707c a ont:BeingBorn ;
    ont:hasAgent res:713527d4-9feb-4015-be1d-6d3dd68b0e89 .

res:713527d4-9feb-4015-be1d-6d3dd68b0e89 rdfs:label "Paul Joseph Heslin" .




In [3]:
# parse open and closed statements (where access has been explicitly granted).

graph_in = rdflib.Graph()
graph_in.parse(pathlib.Path.cwd() / "example.json")

graph_out = rdflib.Graph()
graph_out.bind("ont", ont)
graph_out.bind("res", res)

openstate = [
    s for s, p, o in graph_in.triples((None, rdflib.RDF.type, ont.OpenStatement))
]

for s, p, o in graph_in.triples((None, None, None)):
    if s in openstate and p == ont.hasPayload:
        fragment = rdflib.Graph()
        fragment.parse(data=o)
        graph_out += fragment

with open(pathlib.Path.cwd() / "keys.json") as keys:
    keys = json.load(keys)

allowed = [
    "https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/70596b59-85e3-4fea-81b7-9c3772ebe773",
    "https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/2aea7061-37ff-4581-8d99-327fa210b75d",
    "https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/858f454c-b0a9-4cfb-b280-73204f1fa2b8",
]

allowed_keys = {x: v for x, v in keys.items() if x in allowed}

for s, p, o in graph_in.triples((None, None, None)):
    if str(s) in allowed and p == ont.hasPayload:
        fragment = rdflib.Graph()
        unencruptr = ansible_vault.Vault(allowed_keys[str(s)]).load_raw(o).decode()
        fragment.parse(data=unencruptr)
        graph_out += fragment

print(graph_out.serialize(format="ttl"))


@prefix ont: <https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix res: <https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/> .

res:d4b77081-311b-4a44-b388-6dfb826faa0d a ont:BirthEvent ;
    ont:EventDate "1986-04-14" ;
    ont:EventLocation res:8605ab63-d1d0-4324-8997-8c33de2c3673 ;
    ont:hasActivity res:6dfbacdc-b175-4cd4-8010-7dae5132707c .

res:6dfbacdc-b175-4cd4-8010-7dae5132707c a ont:BeingBorn ;
    ont:hasAgent res:713527d4-9feb-4015-be1d-6d3dd68b0e89 .

res:713527d4-9feb-4015-be1d-6d3dd68b0e89 rdfs:label "Paul Joseph Heslin" .

res:8605ab63-d1d0-4324-8997-8c33de2c3673 rdfs:label "Woden Valley Hospital" .


